In [1]:
from directory_area_verde_treat import (
    definir_diretorio,
    get_data_diretorio
)

project_path = definir_diretorio()
data_path = get_data_diretorio(project_path)

In [2]:
import geopandas as gpd
import pandas as pd
import os

# Àreas Verdes Públicas

As áreas verdes públicas, aqui, são constituídas de:
* Parques Municipais
* Praças e Canteiros
* Terras Indígenas
* (Se eu conseguir pegar, parques)

# Dependências

Este notebook é dependente dos assets resultante dos notebooks '../download_data/malha_parques_municipais', '""/malha_quadra_viaria', '""/malha_terras_indigenas'

In [3]:
assets_path = os.path.join(
    data_path, 
    'assets'
)

gdf_parq_mun = gpd.read_parquet(
    os.path.join(
        assets_path, 
        'parques_municipais.parquet'
    )
)

gdf_quadra = gpd.read_parquet(
    os.path.join(
        assets_path,
        'quadra_viaria_editada.parquet'
    )
)

gdf_t_ind = gpd.read_parquet(
    os.path.join(
        assets_path,
        'terra_indigena.parquet'
    )
)

# Conferir intersecções

Vamos confirmar se parece haver intersecções entre os gdf:

In [ ]:
m= gdf_parq_mun.explore()

gdf_t_ind.explore(m=m, color='orange')

gdf_quadra.explore(m=m, color= 'pink')

# Tratar intersecções

As intersecções podem causar problemas porque se forem ignoradas, aparecerão como duplicatas. Há mais de uma forma de tratá-las.

Aqui, as intersecções serão unidas todas em uma única geometria, na forma de um multipolígono (função dissolve), então serão separadas por polígonos (função explode). OBSTÁCULO: esse novo gdf não terá as informações respectivas a cada polígono, tudo que saberemos sobre eles é a geometria.

In [ ]:
dataframesList = [
    gdf_t_ind[['geometry']],
    gdf_quadra[['geometry']],
    gdf_parq_mun[['geometry']]
]
concat_gdf = gpd.GeoDataFrame(
    pd.concat(
        dataframesList,
        ignore_index=True
    )
)

In [ ]:
concat_gdf.shape

In [ ]:
concat_gdf.sample(3)

In [ ]:
dissolved_gdf = concat_gdf.dissolve()

In [ ]:
gdf_area_verde = dissolved_gdf.explode()

In [ ]:
gdf_area_verde.shape

# Conferir consistência e compatibilidade

In [ ]:
print(
    f"""Conseguimos ver que após o dissolve e o explode, o número de polígonos cai em {
        concat_gdf.shape[0]-gdf_area_verde.shape[0]
    }."""
)

Para confirmarmos se deu tudo certo e essa diferença não significa que perdemos área durante as operações, vamos fazer um overlay do gdf_area_verde com os seus gdfs originários, mas o que retornará disso será apenas a diferença, isto é, quais os trechos do gdf original que ficaram de fora do gdf_area_verde, caso algum tenha ficado.

In [ ]:
confere_parq_mun = gpd.overlay(gdf_parq_mun, gdf_area_verde, how='difference', keep_geom_type=True)

In [ ]:
confere_parq_mun.shape

# Salvar gdf